In [1]:
import cv2
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
if os.environ["KERAS_BACKEND"] =='cntk':
    import cntk
    cntk.device.try_set_default_device(cntk.device.gpu(0)) 
import pandas
import sys
import matplotlib.pyplot as plt
import keras as K
from keras.datasets import mnist
from keras.layers.merge import concatenate
from keras.models import Sequential
from keras.models import Model
from keras.layers import Reshape
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Add
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import scipy

Using TensorFlow backend.


In [2]:
def read_images(filenames,directory,height,width):
    # Read Images into a Numpy array
    
    # Get the array size
    array_size = len(filenames)
    images = np.array([])
    #Preallocate the array.  This is much faster that numpy.append
    images = np.zeros((array_size,height,width))

    i=0
    for file in filenames:
        image = cv2.imread(directory + "/" + file,0)
        # Resize the image.
        resized_image = cv2.resize(image, (width,height))
        # Add it to numpy array
        images[i] = resized_image
        i+=1

    
    
    return images

In [3]:
def get_labels(filelist,labels):
    
    labels.index = labels['Image']
    select_labels = np.array([])
    labels_filenames = []
    for file in filelist:
        select_labels = np.append(select_labels,labels.loc[file]['Encoded_Id'])
        labels_filenames.append(file)
    
    return select_labels,labels_filenames

In [4]:
def encode_labels(labels_df):

    labels = labels_df['Id'].values
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels_encode = le.transform(labels)
    
    labels_df['Encoded_Id'] = labels_encode 
    
    return labels_df

In [5]:
def load_labels(labels):

    
    return train_labels

In [6]:
def load_images(filenames,directory,height,width):
    # Read Images into a Numpy array
    # Get the array size
    array_size = len(filenames)
    images = np.array([])
    #Preallocate the array.  This is much faster that numpy.append
    if os.environ["KERAS_BACKEND"] =='mxnet':
        images = np.zeros((array_size,height,width,3))
    else:
        images = np.zeros((array_size,height,width,1))
    #print("hello")
    i=0
    for file in filenames:
        image = cv2.imread(directory + "/" + file,1)
        # Resize the image.
        resized_image = cv2.resize(image, (width,height))
        # Add it to numpy array
        images[i] = resized_image
        i+=1

    return images

In [7]:
def image_loader(files, labels, batch_size, directory, height, width):

    L = len(files)

    while True:

        batch_start = 0
        batch_end = batch_size
        while batch_start < L:
            limit = min(batch_end, L)

            X = load_images(files[batch_start:limit],directory,height,width)
            
            # Shape data depending on backend
            if os.environ["KERAS_BACKEND"] =='mxnet':
                X = K.utils.to_channels_first(X)
            else: 
                X = X.reshape((X.shape[0],height,width,1))
            
            Y = labels[batch_start:limit]

            #print(X.shape,Y.shape)
            yield (X,Y) #a tuple with two numpy arrays with batch_size sample

            batch_start += batch_size   
            batch_end += batch_size

In [8]:
height=128
width=128

traindir="/home/james/Kaggle/Whale/Data/train/augment"
labelsfile="/home/james/Kaggle/Whale/Data/train.csv"

In [9]:
labels = pandas.read_csv(labelsfile)
#labels.describe()
#with pandas.option_context('display.max_rows', None, 'display.max_columns', None):
#    print(labels['Id'].value_counts())

In [10]:
labels = pandas.read_csv(labelsfile)
y=labels['Id']
values = np.array(y)

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(values),
                                                 values)

#Create Encoded Labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
train_labels = K.utils.to_categorical(integer_encoded)

In [11]:
directory = traindir + "/combined"
validdir = traindir + "/valid"

filenames = os.listdir(directory)
filenames.sort()

#how many files before augmentation
num_orig_filenames = len(os.listdir(traindir + "/1"))
num_aug_filenames = len(filenames)
iterations = int(num_aug_filenames / num_orig_filenames)

augmented_labels = np.array([])
for it in range(iterations):
    if augmented_labels.shape[0] == 0:
        augmented_labels = train_labels
    else:
        augmented_labels = np.append(augmented_labels,train_labels,axis=0)

In [12]:
dropout = 0.5
    
# Get number of classes
num_classes = train_labels.shape[1]
input_shape=(height,width,1)
reshape_size = height * width

In [13]:
# Get number of classes
num_classes = train_labels.shape[1]

if os.environ["KERAS_BACKEND"] =='mxnet':
    input_shape=(3,height,width)
else: 
    input_shape=(height,width,1)

In [14]:
K.backend.image_data_format()

'channels_last'

In [19]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(8, 8)))
model.add(Dropout(dropout))

model.add(Conv2D(128, (8, 8), activation='relu',padding='same'))
model.add(Dropout(dropout))

model.add(Conv2D(64, (2, 2), activation='relu',padding='same'))
model.add(Dropout(dropout))

model.add(Conv2D(32, (2, 2), activation='relu',padding='same'))
model.add(Dropout(dropout))
model.add(Flatten())

model.add(Dense(512, activation='relu'))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Dropout(dropout))


model.add(Dense(num_classes, activation='softmax'))


In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=["accuracy"]
             )


#model.summary()

#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
#Save image_models
filepath="/home/james/Kaggle/Whale/Data/whales-copy" + ".best.hdf5"

checkpoint = ModelCheckpoint(filepath, 
                             monitor="acc",
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')


callbacks_list = [checkpoint]

In [ ]:
epochs=10
batch_size=60

#model.load_weights(filepath)
history = model.fit_generator(
                    image_loader(
                        filenames, augmented_labels, 
                        batch_size, directory, 
                        height, width),                   
                    steps_per_epoch=int(num_aug_filenames/batch_size),  
                    epochs=epochs,
                    callbacks=callbacks_list,
                    #class_weight=class_weights,
                    max_queue_size=120,
                    use_multiprocessing=True,
                    workers=2,
                    verbose=1,
                    shuffle=True)

#mxnet:  

Epoch 1/10


/usr/local/Anaconda/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'
/usr/local/Anaconda/lib/python3.6/site-packages/mxnet/module/bucketing_module.py:408: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.016666666666666666). Is this intended?
  force_init=force_init)


3804/3804 [==============================] - 2572s 676ms/step - loss: 5.8884 - acc: 0.3743

Epoch 00001: acc improved from -inf to 0.37433, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 2/10
 563/3804 [===>..........................] - ETA: 36:24 - loss: 5.6987 - acc: 0.3786

In [ ]:
valid_filenames = os.listdir(validdir)
valid_filenames.sort()

num_val_filenames = len(valid_filenames)
valid_iterations = int(num_val_filenames / num_orig_filenames)
    
valid_labels = np.array([])
for it in range(valid_iterations):
    if valid_labels.shape[0] == 0:
        valid_labels = train_labels
    else:
        valid_labels = np.append(valid_labels,train_labels,axis=0)



In case we want to do validation training later on, we will use a function in Scikit Learn to divide the datasets up.

In [1]:
#X_train, X_test, y_train, y_test = train_test_split(chunk_images,train_labels,
#                                                    test_size=0.33, 
#                                                    random_state=42,
#                                                    shuffle=True)
#print(X_train.shape)
#print(X_test.shape)
#print(y_train.shape)
#print(y_test.shape)

Now that the data is in the proper format and the model has been compiled, the model can be fit.  One can see that the class_weights that were set above have been supplied as an argument to the fit function.  Also notice that the callbacks argument has been initialized to the list that was set above.

In [38]:

del test_images

In [17]:
#del test_images
testdir = "/home/james/Kaggle/Whale/Data/test/4"
testfilenames = os.listdir(testdir)
testfilenames.sort()
testfiles = []

for testfilename in testfilenames:
    testfiles.append(testdir + "/" + testfilename)
    
test_images = read_images(testfiles,"",height,width)
# Reshape Array
test_images = test_images.reshape((test_images.shape[0],height,width,1))


In [21]:
model.load_weights(filepath)
# Make prediction
Predictions = model.predict(test_images)

ValueError: Dimension 0 in both shapes must be equal, but are 3 and 128. Shapes are [3,3,1,128] and [128,3,3,3]. for 'Assign' (op: 'Assign') with input shapes: [3,3,1,128], [128,3,3,3].

In [ ]:
test_csv = "/home/james/Kaggle/Whale/Data/test.csv"

test = testfilenames
col = ['Image']
test_df = pandas.DataFrame(test, columns=col)
test_df['Id'] = ''

for i, pred in enumerate(Predictions):
    ids = label_encoder.inverse_transform(pred.argsort()[-5:][::-1])
    ids = ids.tolist()
    sorted_idx = pred.argsort()[-5:][::-1]
    sorted_values = []
    for idx in sorted_idx:
        sorted_values.append(pred[idx])

    print(sorted_values)
    
    test_df.loc[i, 'Id'] = ' '.join(ids)

print(test_df.head(10))

test_df.to_csv(test_csv, index=False)